In [1]:
import torch
import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
from stock_analyzer import (csv_loader, data_ext)
from stock_analyzer.models import (StockLSTM, StockGRU)

raw_data = csv_loader.load_symbol('./data/raw/', 'TSLA', '1min', '2020')

In [3]:
from torch.utils.data import (TensorDataset, DataLoader)
def create_dataloader(data, lookback, bs):
    x = [data[i:i+lookback, :] for i in range(len(data)-lookback)]
    y = [[data[i, 0]] for i in range(lookback, len(data))]
    ds = TensorDataset(torch.as_tensor(x).float().to(device), torch.as_tensor(y).float().to(device))
    loader = DataLoader(ds, shuffle=True, batch_size=bs)
    return loader

In [4]:
from sklearn.preprocessing import MinMaxScaler
train_scaler = MinMaxScaler()
val_scaler = MinMaxScaler()
test_scaler = MinMaxScaler()

seq_length = 20
batch_size = 256

train_count, val_count, test_count = data_ext.split_counts(raw_data.shape[0], seq_length, train_per=0.7, val_per=0.2)

# 'close' is required and must be on index 0, everything else is optional
features = ['close', 'volume', 'open', 'high', 'low', 'adx', 'ema']
train_data = train_scaler.fit_transform(raw_data[features].values[:train_count+1])
val_data = val_scaler.fit_transform(raw_data[features].values[train_count:train_count+val_count+1])
test_data = test_scaler.fit_transform(raw_data[features].values[train_count+val_count:train_count+val_count+test_count+1])

train_loader = create_dataloader(train_data, seq_length, batch_size)
val_loader = create_dataloader(val_data, seq_length, batch_size)
test_loader = create_dataloader(test_data, seq_length, batch_size)

In [5]:
# Training HyperParameters    
num_epochs = 500
learning_rate = 0.001
print_every = 100

input_size = len(features) #number of features
hidden_size = 200 #number of features in hidden state
num_layers = 2 #number of stacked lstm layers
num_classes = 1 #number of output classes 
fanout_size = 256 # number of fanouts to fully connected layers within the LSTM network

In [6]:
import os
import numpy as np

ckpt_dir = 'stock'
min_valid_loss = float('inf')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
    
# def __init__(self, num_layers, input_size, hidden_size, fanout_size, output_size, dropout_prob, device):
lstm = StockLSTM.StockLSTM(num_layers, input_size, hidden_size, fanout_size, num_classes, 0.25, device) #our lstm class 
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
lstm.to(device)
lstm.train()
for epoch in range(1, num_epochs+1):
    train_losses = []
    val_losses = []
    for x_tensor, y_tensor in train_loader:
        out = lstm.forward(x_tensor)
        optimizer.zero_grad()
        loss = criterion(out, y_tensor)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    lstm.eval()
    for x_tensor, y_tensor in val_loader:
        prediction = lstm.forward(x_tensor)
        val_losses.append(criterion(prediction, y_tensor).item())
        
    # Checkpointing logic
    if np.mean(val_losses) < min_valid_loss:
        min_valid_loss = np.mean(val_losses)
        lstm.save(ckpt_dir + '/lstm_e'+str(int(epoch/1000))+'xxx_lr'+str(learning_rate)+'.pth', np.mean(train_losses), np.mean(val_losses), epoch, learning_rate)
        print("Saving epoch: %d, Training Loss : %1.20f,  Validation Loss: %1.20f ... saving checkpoint %s" % (epoch, np.mean(train_losses), np.mean(val_losses), str(int(epoch/1000))+'xxx'))

    elif epoch % print_every == 0:
        print("Saving epoch: %d, Training Loss : %1.20f,  Validation Loss: %1.20f" % (epoch, np.mean(train_losses), np.mean(val_losses)))
        
    lstm.train()


Saving epoch: 1, Training Loss : 0.02044737454002466814,  Validation Loss: 0.00081080514654466827 ... saving checkpoint 0xxx
Saving epoch: 5, Training Loss : 0.00010991716255883275,  Validation Loss: 0.00077121749495130936 ... saving checkpoint 0xxx
Saving epoch: 10, Training Loss : 0.00007995735037649527,  Validation Loss: 0.00073243111271310498 ... saving checkpoint 0xxx
Saving epoch: 11, Training Loss : 0.00008935371210269338,  Validation Loss: 0.00072863792553795755 ... saving checkpoint 0xxx
Saving epoch: 12, Training Loss : 0.00007858694633507340,  Validation Loss: 0.00069916410044439743 ... saving checkpoint 0xxx
Saving epoch: 16, Training Loss : 0.00007247450457373273,  Validation Loss: 0.00064745137304369658 ... saving checkpoint 0xxx
Saving epoch: 19, Training Loss : 0.00007188589745422269,  Validation Loss: 0.00062652227525423110 ... saving checkpoint 0xxx
Saving epoch: 26, Training Loss : 0.00009006791774728589,  Validation Loss: 0.00061196724198305527 ... saving checkpoint

In [7]:
import os
import numpy as np

ckpt_dir = 'stock'
min_valid_loss = float('inf')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

gru = StockGRU.StockGRU(num_layers, input_size, hidden_size, fanout_size, num_classes, 0.25, device) #our gru class 
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(gru.parameters(), lr=learning_rate)
gru.to(device)
gru.train()
for epoch in range(1, num_epochs+1):
    train_losses = []
    val_losses = []
    for x_tensor, y_tensor in train_loader:
        out = gru.forward(x_tensor)
        optimizer.zero_grad()
        loss = criterion(out, y_tensor)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    gru.eval()
    for x_tensor, y_tensor in val_loader:
        prediction = gru.forward(x_tensor)
        val_losses.append(criterion(prediction, y_tensor).item())
        
    # Checkpointing logic
    if np.mean(val_losses) < min_valid_loss:
        min_valid_loss = np.mean(val_losses)
        gru.save(ckpt_dir + '/gru_e'+str(int(epoch/1000))+'xxx_lr'+str(learning_rate)+'.pth', np.mean(train_losses), np.mean(val_losses), epoch, learning_rate)
        print("Saving epoch: %d, Training Loss : %1.20f,  Validation Loss: %1.20f ... saving checkpoint %s" % (epoch, np.mean(train_losses), np.mean(val_losses), str(int(epoch/1000))+'xxx'))

    elif epoch % print_every == 0:
        print("Saving epoch: %d, Training Loss : %1.20f,  Validation Loss: %1.20f" % (epoch, np.mean(train_losses), np.mean(val_losses)))
        
    gru.train()


Saving epoch: 1, Training Loss : 0.01208971933865118399,  Validation Loss: 0.00052345882176195045 ... saving checkpoint 0xxx
Saving epoch: 2, Training Loss : 0.00012951777752059289,  Validation Loss: 0.00051840140666762312 ... saving checkpoint 0xxx
Saving epoch: 5, Training Loss : 0.00009010950604935588,  Validation Loss: 0.00051274679434435387 ... saving checkpoint 0xxx
Saving epoch: 8, Training Loss : 0.00007743421558061951,  Validation Loss: 0.00045314578282823031 ... saving checkpoint 0xxx
Saving epoch: 10, Training Loss : 0.00007251039967868141,  Validation Loss: 0.00043211140311197251 ... saving checkpoint 0xxx
Saving epoch: 12, Training Loss : 0.00007123847372126911,  Validation Loss: 0.00042322649765205950 ... saving checkpoint 0xxx
Saving epoch: 13, Training Loss : 0.00006681790597043758,  Validation Loss: 0.00042002921376528367 ... saving checkpoint 0xxx
Saving epoch: 14, Training Loss : 0.00006503137851970058,  Validation Loss: 0.00037610564440665012 ... saving checkpoint 0